Resources:
- [Blog NLP](https://medium.com/@nikitasilaparasetty/twitter-sentiment-analysis-for-data-science-using-python-in-2022-6d5e43f6fa6e)

In [1]:
import twint

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# We start configuring what we will scrape
c = twint.Config()

c.Search = ['amazon']       # topic
c.Limit = 10      # number of Tweets to scrape
c.Store_csv = True       # store tweets in a csv file
c.Output = "amazon_tweets.csv"     # path to csv file

twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
